### **[LeetCode Link](https://leetcode-cn.com/problems/reachable-nodes-in-subdivided-graph/solution/xi-fen-tu-zhong-de-ke-dao-da-jie-dian-by-leetcode/)**

## Dijkstra 算法
### 思路
将原始图作为加权无向图处理，我们可以使用 *$Dijkstra$* 算法查找原始图中的所有可到达结点。 但是，这不足以解决仅部分使用细分边的示例。

当我们沿着边（沿任一方向）行进时，我们可以跟踪我们使用它的程度。最后，我们想知道我们在原始图中到达的每个结点，以及每条边的利用率之和。
### 算法
* 我们使用 *$Dijkstra$* 算法 来找出从源到所有目标的最短距离。 
* 另外，对于每条（有向）边 `(node，nei)`，我们将跟踪有多少新结点（从原始边细分而来的新结点）被使用。 最后，我们将总结每条边的利用率。

### 复杂度分析
* 时间复杂度：$\mathcal{O}(E \log N)$，其中 $E$ 是 $edges$ 的长度。
* 空间复杂度：$\mathcal{O}(N)$。

In [17]:
class Solution(object):
    def reachableNodes(self, edges, M, N):
        graph = collections.defaultdict(dict)
        for u, v, w in edges:
            graph[u][v] = graph[v][u] = w
#         print(graph)
        pq = [(0, 0)]
        dist = {0: 0}
        used = {}
        ans = 0

        while pq:
            d, node = heapq.heappop(pq)
#             print("d_node: {} {}".format(d, node))
            if d > dist[node]: continue
            # Each node is only visited once.  We've reached
            # a node in our original graph.
            ans += 1
            
            for nei, weight in graph[node].items():
#                 print("-"*20)
#                 print(nei,weight)
                # M - d is how much further we can walk from this node;
                # weight is how many new nodes there are on this edge.
                # v is the maximum utilization of this edge.
                v = min(weight, M - d)
                used[node, nei] = v
#                 print(used)
                # d2 is the total distance to reach 'nei' (nei***or) node
                # in the original graph.
                d2 = d + weight + 1
                if d2 < dist.get(nei, M+1):
                    heapq.heappush(pq, (d2, nei))
                    dist[nei] = d2
#                     print(pq)
#                     print(dist)

        # At the end, each edge (u, v, w) can be used with a maximum
        # of w new nodes: a max of used[u, v] nodes from one side,
        # and used[v, u] nodes from the other.
        for u, v, w in edges:
            ans += min(w, used.get((u, v), 0) + used.get((v, u), 0))

        return ans

In [18]:
import collections, heapq
solution = Solution()
solution.reachableNodes([[0,1,10],[0,2,1],[1,2,2]], 6, 3)

13

In [ ]:
class Solution {
    public int reachableNodes(int[][] edges, int M, int N) {
        Map<Integer, Map<Integer, Integer>> graph = new HashMap();
        for (int[] edge: edges) {
            int u = edge[0], v = edge[1], w = edge[2];
            graph.computeIfAbsent(u, x->new HashMap()).put(v, w);
            graph.computeIfAbsent(v, x->new HashMap()).put(u, w);
        }

        PriorityQueue<ANode> pq = new PriorityQueue<ANode>(
                (a, b) -> Integer.compare(a.dist, b.dist));
        pq.offer(new ANode(0, 0));

        Map<Integer, Integer> dist = new HashMap();
        dist.put(0, 0);
        Map<Integer, Integer> used = new HashMap();
        int ans = 0;

        while (!pq.isEmpty()) {
            ANode anode = pq.poll();
            int node = anode.node;
            int d = anode.dist;

            if (d > dist.getOrDefault(node, 0)) continue;
            // Each node is only visited once.  We've reached
            // a node in our original graph.
            ans++;

            if (!graph.containsKey(node)) continue;
            for (int nei: graph.get(node).keySet()) {
                // M - d is how much further we can walk from this node;
                // weight is how many new nodes there are on this edge.
                // v is the maximum utilization of this edge.
                int weight = graph.get(node).get(nei);
                int v = Math.min(weight, M - d);
                used.put(N * node + nei, v);

                // d2 is the total distance to reach 'nei' (nei***or) node
                // in the original graph.
                int d2 = d + weight + 1;
                if (d2 < dist.getOrDefault(nei, M+1)) {
                    pq.offer(new ANode(nei, d2));
                    dist.put(nei, d2);
                }
            }
        }

        // At the end, each edge (u, v, w) can be used with a maximum
        // of w new nodes: a max of used[u, v] nodes from one side,
        // and used[v, u] nodes from the other.
        // [We use the encoding (u, v) = u * N + v.]
        for (int[] edge: edges) {
            ans += Math.min(edge[2], used.getOrDefault(edge[0] * N + edge[1], 0) +
                    used.getOrDefault(edge[1] * N + edge[0], 0) );
        }

        return ans;
    }

    class ANode {
        int node, dist;
        ANode(int n, int d) {
            node = n;
            dist = d;
        }
    }
}